In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from IPython.display import clear_output
import pickle

#common utilities for transforming data
from commons import Utilities
utils = Utilities()

In [2]:
print("Finding medians of columns in chunks...")
medians=[]

filename='../input/train_numeric.csv'
chunknum = 0
for chunk in pd.read_csv(filename, header=0, chunksize=200000):
    median_chunk = Utilities.findNa(utils,chunk,chunknum)
    medians.append(median_chunk)
    chunknum = chunknum +1
    clear_output()
    print("%d percent" % int((chunknum*200000/1183748)*100))
    
medians = pd.DataFrame(medians)


101 percent


In [4]:
pickle.dump( medians, open( "../exported/medians.p", "wb" ) )

In [3]:
#Train
models=[]
def train(df):
    print("Filling missing data with median values...")
    df = utils.normalize_data_w_medians(df,medians)
    df['Id2'] = df['Id'] 
    print("Preparing data for training...")
    X = df.drop(['Id','Response'],axis=1)
    y = df['Response']
    
    # Random Forests 
    model = RandomForestClassifier(n_estimators=100)
    print("Training...")
    model.fit(X,y)
    models.append(model)

print("Reading train data into memory in chunks...")
chunknum = 0
for chunk in pd.read_csv('../input/train_numeric.csv', header=0, chunksize=200000):
    train(chunk)
    chunknum = chunknum +1
    clear_output()
    print("%d percent" % int((chunknum*200000/1183748)*100))
pickle.dump( models, open( "../exported/models.p", "wb" ) )

101 percent
